In [ ]:
# === Pacotes necessários / Required packages ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from itertools import combinations

# Sem classe indexada / No indexed class

In [ ]:
# ===== 1. Ler os dados .XLSX / Read .XLSX data =====
dados = pd.read_excel("NAME.xlsx")
print(dados.head())

In [ ]:
# === Ler os dados .CSV / Read .CSV data ===
dados = pd.read_csv('NAME.csv', sep=';')  # especifica o separador / specifies the separator
print(dados.head())

In [ ]:
# === Padronização dos dados / Data standardization ===
scaler = StandardScaler()
dados_padronizados = scaler.fit_transform(dados)

In [ ]:
# === PCA ===
pca = PCA()
pca.fit(dados_padronizados)

# Autovetores / Eigenvectors (loadings)
autovetores = pca.components_.T
# Autovalores (variância explicada em valores absolutos) / Eigenvalues ​​(variance explained in absolute values)
explained_variances = pca.explained_variance_
# Proporção da variância explicada (%) / Proportion of variance explained (%)
variancia_explicada_pct = pca.explained_variance_ratio_

# === Scree Plot ===
plt.figure()
plt.plot(np.arange(1, len(explained_variances)+1), explained_variances, 'o-', color='blue')
plt.title("Scree Plot")
plt.xlabel("Principal Component")
plt.ylabel("Eigenvalues")
plt.grid(True)
plt.show()

In [ ]:
# === Biplot (PC1 vs PC2) ===
scores = pca.transform(dados_padronizados)
plt.figure(figsize=(7,6))
plt.scatter(scores[:,0], scores[:,1], alpha=0.6, c="steelblue")

for i, var in enumerate(dados.columns):
    plt.arrow(0, 0, autovetores[i,0]*max(scores[:,0]), autovetores[i,1]*max(scores[:,1]),
              color="red", alpha=0.7, head_width=0.1)
    plt.text(autovetores[i,0]*max(scores[:,0])*1.1,
             autovetores[i,1]*max(scores[:,1])*1.1,
             var, color="red", ha="center", va="center")

plt.title("Biplot PCA")
plt.xlabel(f"PC1 ({variancia_explicada_pct[0]*100:.2f}%)")
plt.ylabel(f"PC2 ({variancia_explicada_pct[1]*100:.2f}%)")
plt.grid(True)
plt.show()

In [ ]:
# === Seleção de componentes / Component selection ===
num_componentes = 6 # muda aqui o número de PCs / change the number of PCs here
autovetores_selecionados = autovetores[:, :num_componentes]

# Projeção dos dados padronizados / Projection of standardized data
dados_projetados_pca = dados_padronizados @ autovetores_selecionados
dados_projetados_pca_df = pd.DataFrame(
    dados_projetados_pca,
    columns=[f"PC{i+1}" for i in range(num_componentes)]
)

print(dados_projetados_pca_df)

In [ ]:
# === Gráficos de dispersão entre PCs / Scatterplots between PCs ===
componentes = [f"PC{i}" for i in range(1, 6)] # muda aqui o número de PCs / change the number of PCs here
from itertools import combinations
combinacoes = list(combinations(componentes, 2))

for x_comp, y_comp in combinacoes:
    x_index = int(x_comp.replace("PC", ""))
    y_index = int(y_comp.replace("PC", ""))

    plt.figure(figsize=(6,5))
    sns.scatterplot(data=dados_projetados_pca_df, x=x_comp, y=y_comp, s=50, color="steelblue", alpha=0.6)
    sns.kdeplot(data=dados_projetados_pca_df, x=x_comp, y=y_comp, color="darkred", linewidth=0.7)

    plt.title("PCA Visualization: Data Structure")
    plt.xlabel(f"Principal Component {x_index} ({variancia_explicada_pct[x_index-1]*100:.2f}%)")
    plt.ylabel(f"rincipal Component {y_index} ({variancia_explicada_pct[y_index-1]*100:.2f}%)")
    plt.show()

In [ ]:
# === Loadings ===
loadings_df = pd.DataFrame(
    autovetores,
    index=dados.columns,
    columns=[f"PC{i+1}" for i in range(len(autovetores[0]))]
)

print(loadings_df)

# Salvar loadings em CSV e Excel / Save loadings in CSV and Excel
loadings_df.to_csv("pca_loadings.csv")
loadings_df.to_excel("pca_loadings.xlsx")

print("pca_loadings.csv and pca_loadings.xlsx saved.")

In [ ]:
# === Variância explicada / Variance explained ===
explained_variance_ratio = variancia_explicada_pct
cumulative_explained_variance = np.cumsum(explained_variance_ratio)

variance_table = pd.DataFrame({
    "Principal_Component": np.arange(1, len(explained_variance_ratio)+1),
    "Proportion_of_Variance": explained_variance_ratio,
    "Cumulative_Proportion": cumulative_explained_variance
})

print(variance_table)

variance_table.to_csv("variance_table.csv", index=False)
variance_table.to_excel("variance_table.xlsx", index=False)

print("variance_table.xlsx saved.")
print("variance_table.csv saved.")


## Reduzindo a dimensionalidade / Reducing dimensionality



In [ ]:
# Escolha quantos PCs você quer considerar / Choose how many PCs you want to consider
n_pcs = 6  # <<--- altere esse valor conforme necessário / change this value as needed

# Calcula a soma dos quadrados dos loadings / Calculates the sum of the squares of the loadings
pcs_de_interesse = [f'PC{i+1}' for i in range(n_pcs)]
contribuicoes = (loadings_df[pcs_de_interesse] ** 2).sum(axis=1)

# Ordena por importância / Sort by importance
contribuicoes_ordenadas = contribuicoes.sort_values(ascending=False)

# Exibe o top N / Displays the top N
top_n = 6  # <<--- altere esse valor conforme necessário / change this value as needed
print(f"\nTop {top_n} most important variables (based on {n_pcs} first PCs):")
top_variaveis = contribuicoes_ordenadas.head(top_n)
print(contribuicoes_ordenadas.head(top_n))

In [ ]:
# Limpa o DataFrame original para manter apenas as variáveis selecionadas / Clears the original DataFrame to keep only the selected variables
dados_filtrados = dados[top_variaveis.index.tolist()]

# (Opcional) Substituir o DataFrame original, se quiser sobrescrever: / (Optional) Replace the original DataFrame, if you want to overwrite it:
# dados = dados_filtrados

# Exibe as primeiras linhas do novo DataFrame / Displays the first few rows of the new DataFrame
print("\nDataFrame 'data' after filtering by the most important variables:")
print(dados_filtrados.head())

In [ ]:
# Salvar os dados filtrados em um arquivo Excel / Save the filtered data to an Excel file
dados_filtrados.to_excel('dados_filtrados.xlsx', index=False)

print("✅ File 'data_filtrados.xlsx' saved successfully.")

In [ ]:
# Salvar os dados filtrados em um arquivo CSV / Save the filtered data to a CSV file
dados_filtrados.to_csv('dados_filtrados.csv', index=False)

print("✅ File 'data_filtrados.csv' saved successfully.")

# Com classe indexada / With indexed class

In [ ]:
# ===== 1. Ler os dados .XLSX / Read .XLSX data =====
dados = pd.read_excel("NAME.xlsx")
print(dados.head())

In [ ]:
# === Ler os dados .CSV / Read .CSV data ===
dados = pd.read_csv('NAME.csv', sep=',')  # especifica o separador / specifies the separator
print(dados.head())

In [ ]:
# Separar classes (primeira coluna) dos preditores / Separate classes (first column) of predictors
classe = dados.iloc[:, 0]     # primeira coluna = classes / first column = classes
X = dados.iloc[:, 1:]         # resto = variáveis numéricas / other = numeric variables

print("Classes detected:\n", classe.unique())
print("\nSamples:\n", X.head())

In [ ]:
# === Padronização / Standardization ===
scaler = StandardScaler()
X_padronizado = scaler.fit_transform(X)

In [ ]:
# === PCA ===
pca = PCA()
pca.fit(X_padronizado)

# Autovetores / Eigenvectors (loadings)
autovetores = pca.components_.T
explained_variances = pca.explained_variance_
variancia_explicada_pct = pca.explained_variance_ratio_

# === Scree Plot ===
plt.figure()
plt.plot(np.arange(1, len(explained_variances)+1), explained_variances, 'o-', color='blue')
plt.title("Scree Plot")
plt.xlabel("Componente Principal")
plt.ylabel("Autovalor")
plt.grid(True)
plt.show()

In [ ]:
# === Biplot (PC1 vs PC2) ===
scores = pca.transform(X_padronizado)
plt.figure(figsize=(7,6))

# cor dos pontos pela classe / color of points by class
sns.scatterplot(x=scores[:,0], y=scores[:,1], hue=classe, palette="tab10", alpha=0.7)

for i, var in enumerate(X.columns):
    plt.arrow(0, 0, autovetores[i,0]*max(scores[:,0]), autovetores[i,1]*max(scores[:,1]),
              color="red", alpha=0.7, head_width=0.1)
    plt.text(autovetores[i,0]*max(scores[:,0])*1.1,
             autovetores[i,1]*max(scores[:,1])*1.1,
             var, color="red", ha="center", va="center")

plt.title("Biplot PCA")
plt.xlabel(f"PC1 ({variancia_explicada_pct[0]*100:.2f}%)")
plt.ylabel(f"PC2 ({variancia_explicada_pct[1]*100:.2f}%)")
plt.legend(title="Class")
plt.grid(True)
plt.show()

In [ ]:
# === Seleção de componentes / Component selection ===
num_componentes = 20    # <<--- altere esse valor conforme necessário / change this value as needed
autovetores_selecionados = autovetores[:, :num_componentes]
X_proj = X_padronizado @ autovetores_selecionados

dados_proj_pca_df = pd.DataFrame(
    X_proj,
    columns=[f"PC{i+1}" for i in range(num_componentes)]
)
dados_proj_pca_df["Classe"] = classe.values  # adiciona classes / add classes

print(dados_proj_pca_df.head())

In [ ]:
# === Gráficos de dispersão entre PCs / Scatterplots between PCs ===
componentes = [f"PC{i}" for i in range(1, 6)]
combinacoes = list(combinations(componentes, 2))

for x_comp, y_comp in combinacoes:
    x_index = int(x_comp.replace("PC", ""))
    y_index = int(y_comp.replace("PC", ""))

    plt.figure(figsize=(6,5))
    sns.scatterplot(data=dados_proj_pca_df, x=x_comp, y=y_comp,
                    hue="Classe", palette="tab10", s=50, alpha=0.7)

    plt.title("PCA Visualization: Data Structure")
    plt.xlabel(f"Principal Component {x_index} ({variancia_explicada_pct[x_index-1]*100:.2f}%)")
    plt.ylabel(f"Principal Component {y_index} ({variancia_explicada_pct[y_index-1]*100:.2f}%)")
    plt.legend(title="Class")
    plt.show()

In [ ]:
# === Loadings ===
loadings_df = pd.DataFrame(
    autovetores,
    index=X.columns,
    columns=[f"PC{i+1}" for i in range(len(autovetores[0]))]
)

print(loadings_df)

loadings_df.to_csv("pca_loadings.csv")
loadings_df.to_excel("pca_loadings.xlsx")
print("pca_loadings.csv and pca_loadings.xlsx saved.")


In [ ]:
# === Variância explicada / Variance explained ===
explained_variance_ratio = variancia_explicada_pct
cumulative_explained_variance = np.cumsum(explained_variance_ratio)

variance_table = pd.DataFrame({
    "Principal_Component": np.arange(1, len(explained_variance_ratio)+1),
    "Proportion_of_Variance": explained_variance_ratio,
    "Cumulative_Proportion": cumulative_explained_variance
})

print(variance_table)

variance_table.to_csv("variance_table.csv", index=False)
variance_table.to_excel("variance_table.xlsx", index=False)
print("variance_table.xlsx saved.")

## Reduzindo a dimensionalidade

In [ ]:
# Escolha quantos PCs você quer considerar / Choose how many PCs you want to consider
n_pcs = 20 # <<--- altere esse valor conforme necessário / change this value as needed

# Calcula a soma dos quadrados dos loadings / Calculates the sum of the squares of the loadings
pcs_de_interesse = [f'PC{i+1}' for i in range(n_pcs)]
contribuicoes = (loadings_df[pcs_de_interesse] ** 2).sum(axis=1)

# Ordena por importância / Sort by importance
contribuicoes_ordenadas = contribuicoes.sort_values(ascending=False)

# Exibe o top N / Displays the top N
top_n = 20  # <<--- altere esse valor conforme necessário / change this value as needed
print(f"\nTop {top_n} most important variables (based on {n_pcs} first PCs):")
top_variaveis = contribuicoes_ordenadas.head(top_n)
print(contribuicoes_ordenadas.head(top_n))

In [ ]:
# Limpa o DataFrame original para manter apenas as variáveis selecionadas / Clears the original DataFrame to keep only the selected variables
dados_filtrados = dados[top_variaveis.index.tolist()]

# (Opcional) Substituir o DataFrame original, se quiser sobrescrever: / (Optional) Replace the original DataFrame, if you want to overwrite it:
# dados = dados_filtrados

# Adicionar a coluna 'Classe' de volta como a primeira coluna / Add the 'Class' column back as the first column
dados_filtrados.insert(0, 'Classe', classe)

# Exibe as primeiras linhas do novo DataFrame / Displays the first few rows of the new DataFrame
print("\nDataFrame 'data' after filtering by the most important variables:")
print(dados_filtrados.head())

In [ ]:
# Salvar os dados filtrados em um arquivo Excel / Save the filtered data to an Excel file
dados_filtrados.to_excel('dados_filtrados.xlsx', index=False)

print("✅ File 'data_filtrados.xlsx' saved successfully.")

In [ ]:
# Salvar os dados filtrados em um arquivo CSV / Save the filtered data to a CSV file
dados_filtrados.to_csv('dados_filtrados.csv', index=False)

print("✅ File 'data_filtrados.csv' saved successfully.")